In [2]:
# -*- coding: UTF-8 -*-

""" Code to scrap Lyon-Bron weather from a specific date """

from lxml import html
from datetime import datetime, timedelta
from cleaner import cleaner

import requests
import pandas as pd

In [75]:
# Defines which date you want to get data from, january = 1, december = 12
startingDate = datetime(2016,8,1)
endingDate = datetime(2016,8,31)
deltaDate = endingDate - startingDate

# Defines which weather station code (For Lyon-Bron it is 7480)
cityCode = 7480

# Initialisation
body = []
header = []
i = 0

for d in range(0, deltaDate.days+1):
    sourcingDate = startingDate + timedelta(days=d)

    # Get webpage source code to scrap data from it with our parameters
    rawSource = requests.get("http://www.meteociel.fr/temps-reel/obs_villes.php?jour2=%s&mois2=%s&annee2=%s&code2=%s" \
                             % (sourcingDate.day, sourcingDate.month-1, sourcingDate.year, cityCode)) 

    # Cleaning raw source code
    source = cleaner(rawSource.text)

    # Transforming into a tree
    tree = html.fromstring(source)
    # Browsing tree, seeking for tr 
    tables = tree.xpath('//tr')
    
    # Initializing counter
    k = 0
        
    # Get header
    header = tables[0].xpath('td/text()')
    
    # Getting values and refactoring date/time
    for td in tables:
        if k > 0:
            body.append(tables[k].xpath('td/text()'))
            body[i][0] = body[i][0].lower()
            addHour = body[i][0]
            newDate = sourcingDate + timedelta(hours=int(addHour))
            
            # Take care of summer/winter UTC time for 2015/2016 
            # Winter 2016
            if newDate > datetime(2016,10,30,2):
                addUTC = "+01"
            # Summer 2016
            elif newDate > datetime(2016,3,27,2):
                addUTC = "+02"
            # Winter 2015
            elif newDate > datetime(2015,10,25,2):
                addUTC = "+01"
            # DIY
            else:
                addUTC = "Z"
            
            newDate = str(newDate).replace(" ", "")
            newDate = newDate[:10] + "T" + newDate[10:] + addUTC
            body[i][0] = newDate           
            i+=1
        k+=1

In [76]:
header[0] = "Datetime"
data = pd.DataFrame(body, columns=header)
data = data.drop('Temps', 1)
print(data)
data.to_json("test.json", orient="records")

                   Datetime Neb. Visi Temperature Humidite Humidex Windchill  \
0    2016-08-01T23:00:00+02        60        21.3      57    23.7       21.3   
1    2016-08-01T22:00:00+02        60        22.6      51    24.7       22.6   
2    2016-08-01T21:00:00+02        60        23.7      48    25.9       23.7   
3    2016-08-01T20:00:00+02        45        26.3      38    27.9       26.3   
4    2016-08-01T19:00:00+02        40          27      35    28.3         27   
5    2016-08-01T18:00:00+02        50        27.5      36    29.2       27.5   
6    2016-08-01T17:00:00+02        50        27.5      33    28.6       27.5   
7    2016-08-01T16:00:00+02        60        26.8      35    28.1       26.8   
8    2016-08-01T15:00:00+02        60        26.6      35    27.8       26.6   
9    2016-08-01T14:00:00+02        60        25.5      42    27.5       25.5   
10   2016-08-01T13:00:00+02        60        25.4      42    27.4       25.4   
11   2016-08-01T12:00:00+02        50   